## 全连接神经网络
处理文本分类问题
1. 原始文本转换成tfidf特征向量
2. 将特征向量feed给全连接网络

keras

In [1]:
from __future__ import print_function

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

import numpy as np 
import pandas as pd 


Using TensorFlow backend.


In [2]:
batch_size = 4000
num_classes = 19
epochs = 10

读取数据
drop掉不用的信息
把词转换成特征向量
hua

In [3]:

# ~/Downloads/train_set.csv
# ~/workspace/sublime/daguan/train_sample.csv
print('read data')
df_train = pd.read_csv('~/workspace/sublime/daguan/train_sample.csv')
df_test = pd.read_csv('~/workspace/sublime/daguan/train_sample.csv')

df_train.drop(columns=['article','id'], inplace=True)
df_test.drop(columns=['article'], inplace=True)

print('feature:')
# 把文章转换为tfidf向量
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, use_idf=True,smooth_idf=True, sublinear_tf=True,max_features=100000)
vectorizer.fit(df_train['word_seg'])



# 训练的时候只用到词
x_train = vectorizer.transform(df_train['word_seg'])
y_train = df_train['class'] - 1
x_test = vectorizer.transform(df_test['word_seg'])




train_X,test_X, train_y, test_y = train_test_split(x_train,
                                                   y_train,
                                                   test_size = 0.2,
                                                   random_state = 0)

# 把cat转换成onehot
train_y = keras.utils.to_categorical(train_y, num_classes)
test_y = keras.utils.to_categorical(test_y, num_classes)

print(train_y)
print(train_X.shape)

read data
feature:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
(8, 419)


In [ ]:

# 100000->10000->1000->100->19
# 模型 x_train.shape[1]
model = Sequential()
model.add(Dense(10000, activation='relu', input_shape=(train_X.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(train_X, train_y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(test_X, test_y))

model.save('mlp.h5')

# 评价
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
